In [221]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [60]:
import joblibspark

In [62]:
import numpy as np

In [222]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [223]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import MinMaxScaler

In [5]:
import mlflow
from mlflow.tracking import MlflowClient

In [6]:
import os

# Установите переменные окружения AWS_ACCESS_KEY_ID и AWS_SECRET_ACCESS_KEY
# os.environ['AWS_ACCESS_KEY_ID'] = 'YCAJEExfyMKWDYUc36MrS3BEy'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'YCN2-TEIpayiu9UXNt-_kXFsnEsSqxQOJX9ZO3mM'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'https://storage.yandexcloud.net/'
os.environ['MLFLOW_TRACKING_URI'] = 'http://158.160.36.33:8000' 

In [63]:
seed = 42
np.random.seed(42)

In [225]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.set("spark.executor.memoryOverhead", "2048m")  # Увеличьте значение памяти надежности исполнителя
conf.set("spark.yarn.executor.memoryOverhead", "2048m")  # Увеличьте значение памяти надежности исполнителя для YARN
conf.set("spark.yarn.scheduler.maximum-allocation-mb", "16384")  # Увеличьте максимальное выделение памяти для исполнителей
conf.set("spark.yarn.nodemanager.resource.memory-mb", "20480")  # Увеличьте общее количество памяти на датаноде

sc = SparkContext(conf=conf)

In [7]:
spark = (
    SparkSession
    .builder
    .appName("antiFraud")
#     .config("spark.executor.cores", "8")  # Установите значение равным количеству ядер на датаноде (8 в вашем случае)
#     .config("spark.executor.memory", "28g")  # Установите значение, учитывая доступную память на датаноде и резервируемую память
#     .config("spark.executor.instances", "16")  # Установите значение, учитывая доступные ресурсы на кластере
#     .config("spark.default.parallelism", "24")  # Установите значение равным количеству доступных ядер на кластере (24 в вашем случае)
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.ui.port", "4040")
    .getOrCreate()
)

In [226]:
spark = (SparkSession
    .builder
    .appName("aFraud")
    .master("yarn")
    .config("spark.executor.instances", "3")
    .config("spark.executor.cores", "4")
    .config("spark.executor.memory", "12")
#     .config("spark.default.parallelism", "24")
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:MaxGCPauseMillis=200")
    .getOrCreate()
)


In [227]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [76]:
! hadoop distcp s3a://fraudstop/data_clean/df2.parquet /user/data_clean

2024-01-03 17:40:00,161 INFO tools.DistCp: Input Options: DistCpOptions{atomicCommit=false, syncFolder=false, deleteMissing=false, ignoreFailures=false, overwrite=false, append=false, useDiff=false, useRdiff=false, fromSnapshot=null, toSnapshot=null, skipCRC=false, blocking=true, numListstatusThreads=0, maxMaps=20, mapBandwidth=0.0, copyStrategy='uniformsize', preserveStatus=[], atomicWorkPath=null, logPath=null, sourceFileListing=null, sourcePaths=[s3a://fraudstop/data_clean/df2.parquet], targetPath=/user/data_clean, filtersFile='null', blocksPerChunk=0, copyBufferSize=8192, verboseLog=false, directWrite=false}, sourcePaths=[s3a://fraudstop/data_clean/df2.parquet], targetPathExists=true, preserveRawXattrsfalse
2024-01-03 17:40:00,433 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-2dnw3p4s8fl6b5t7.mdb.yandexcloud.net/10.128.0.30:8032
2024-01-03 17:40:00,614 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-2dnw3p4s8fl6b5t7.mdb.yand

In [228]:
# df = (spark.read.format('parquet').load("s3a://fraudstop/data_clean/df.parquet"))
df = spark.read.parquet("/user/data_clean/df.parquet")

In [137]:
df

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
7024176,2019-08-26 17:40:48,484699,942,87.75,409248,4,0,0
7024183,2019-08-26 18:16:29,484704,88,132.63,411389,4,0,0
7025110,2019-08-26 08:43:12,485299,385,14.35,376992,4,0,0
7025517,2019-08-26 10:40:09,485558,0,74.46,384009,4,0,0
7025711,2019-08-26 10:52:21,485697,117,13.22,384741,4,0,0
7025840,2019-08-26 12:03:30,485769,448,69.9,389010,4,0,0
7025999,2019-08-26 12:06:17,485872,585,61.53,389177,4,0,0
7026025,2019-08-26 10:26:52,485887,276,41.58,383212,4,0,0
7026180,2019-08-26 04:51:41,485991,310,5.31,363101,4,0,0
7026287,2019-08-26 20:19:30,486070,781,45.9,418770,4,0,0


In [200]:
mlflow.get_tracking_uri()

'http://158.160.36.33:8000'

In [12]:
client = MlflowClient()
experiment_id = client.create_experiment("antifraud_notebook")
experiment_id

'1'

In [13]:
experiment = client.get_experiment_by_name("antifraud_notebook")
print("Experiment Name:", experiment.name)
print("Experiment ID:", experiment.experiment_id)
print("Created Time:", experiment.creation_time)

Experiment Name: antifraud_notebook
Experiment ID: 1
Created Time: 1704530013367


In [82]:
client.search_experiments()

[<Experiment: artifact_location='s3://fraudstop/artifacts/1', creation_time=1704299135126, experiment_id='1', last_update_time=1704299135126, lifecycle_stage='active', name='antiFraud-ml', tags={}>,
 <Experiment: artifact_location='s3://fraudstop/artifacts/0', creation_time=1704294580264, experiment_id='0', last_update_time=1704294580264, lifecycle_stage='active', name='Default', tags={}>]

In [16]:
# experiment_id = client.create_experiment("testmlflow")

In [17]:
# experiment = client.get_experiment_by_name("testmlflow")
# print("Experiment Name:", experiment.name)
# print("Experiment ID:", experiment.experiment_id)
# print("Created Time:", experiment.creation_time)


Experiment Name: testmlflow
Experiment ID: 1
Created Time: 1704215903651


In [14]:
# name = "SocialMediaTextAnalyzer"
# tags = {"nlp.framework": "Spark NLP"}
# desc = "This sentiment analysis model classifies the tone-happy, sad, angry."
# client.create_registered_model(name, tags, desc)

<RegisteredModel: aliases={}, creation_timestamp=1704324972123, description='This sentiment analysis model classifies the tone-happy, sad, angry.', last_updated_timestamp=1704324972123, latest_versions=[], name='SocialMediaTextAnalyzer', tags={'nlp.framework': 'Spark NLP'}>

# FE

В нашем датасете признаков для обучения модели немного, начну с тех что есть customer_id, terminal_id, tx_amount
и добавлю время суток, для этог посчитаю кол-во минут от начала суток, есть сомнения насчёт customer_id

In [229]:
data = df.select('customer_id', 'terminal_id', 'tx_amount', 'tx_time_seconds', 'tx_fraud')

In [58]:
print(data.dtypes)

[('timestamp', 'timestamp'), ('customer_id', 'int'), ('terminal_id', 'int'), ('tx_amount', 'double'), ('tx_fraud', 'int'), ('hour_from_start', 'int')]


In [89]:
data

customer_id,terminal_id,tx_amount,tx_time_seconds,tx_fraud
127,327,15.43,10068,0
251,284,100.68,68959,0
419,844,67.96,52102,1
691,80,7.17,23688,0
851,665,27.15,52395,0
1107,109,13.64,33870,0
1155,225,110.84,28106,0
1278,563,5.97,15831,0
1316,450,5.5,40149,1
1557,710,77.72,44870,0


In [52]:
# data = data.withColumn("timestamp", F.to_timestamp(data["tx_datetime"], "yyyy-MM-dd HH:mm:ss"))
# df = df.withColumn("timestamp", to_timestamp(df["timestamp"], "yyyy-MM-dd HH:mm:ss"))
# Создание колонки "minutes_from_start" с количеством минут от начала суток
# data = data.withColumn("minutes_from_start", F.minute(data["timestamp"]))

In [57]:
# data = data.withColumn("hour_from_start", F.hour(data["timestamp"]))

In [85]:
data.describe()

summary,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_fraud
count,45683790,45683790,45683790,45683790,45683790
mean,500449.0352239602,26601.728341168717,50.78766711628015,1296073.8352382104,0.05236172392877211
stddev,288530.40631089243,1528269.3368947941,35.25709865412637,748058.7987241015,0.222755414933374
min,0,0,0.0,0,0
max,999999,89518096,146.99,2592000,1


In [230]:
FEATURES = ['customer_id', 'terminal_id', 'tx_amount', 'tx_time_seconds']

In [80]:
# data = data.withColumn("minutes_from_start", col("minutes_from_start").cast("double"))

In [28]:
# print(train_data.dtypes)

[('minutes_from_start', 'int'), ('customer_id', 'int'), ('terminal_id', 'int'), ('tx_amount', 'double'), ('tx_fraud', 'int'), ('features', 'vector')]


In [231]:
assembler = VectorAssembler()\
    .setInputCols(['customer_id', 'terminal_id', 'tx_amount', 'tx_time_seconds'])\
    .setOutputCol("features")

train_data = assembler.transform(data)

In [232]:
# формируем трансформер, передаем входные и выходные столбцы 
scaler = MinMaxScaler()\
    .setInputCol("features")\
    .setOutputCol("scaledFeatures")

# преобразовываем
train_data = scaler.fit(train_data).transform(train_data)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f00e0286b80>
Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'RandomForestClassifier' object has no attribute '_java_obj'


In [233]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [234]:
splits = train_data.randomSplit([0.9, 0.1], 24)
training = splits[0]
test = splits[1]

In [24]:
rf = RandomForestClassifier(labelCol="tx_fraud", featuresCol="features", numTrees=40, seed=seed )

In [158]:
# training

In [25]:
evaluator = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', metricLabel=1, beta=1.0) 

In [26]:
run_name = 'RF_baseline'

with mlflow.start_run(run_name=run_name, experiment_id=experiment.experiment_id):
    mlflow.log_param('features', FEATURES)
    mlflow.log_param('numTrees', 40)
    
    model = rf.fit(training)
    
    evaluator = RegressionEvaluator(labelCol='tx_fraud')
    predictions = model.transform(test)
    F1 = evaluator.evaluate(predictions)
    
    mlflow.log_metric("F1", F1)
    mlflow.spark.save_model(model, "random_forest")

    mlflow.spark.log_model(model, "random_forest")

In [30]:
metrics = MulticlassMetrics(predictionAndLabels)

0.23237575345271957

In [45]:
evaluator = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', 
                                              metricLabel=1, beta=1.0, labelCol='tx_fraud') 
f1_score = evaluator.evaluate(predictions)
f1_score

0.0

In [46]:
evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR', labelCol='tx_fraud') 
areaUnderPR = evaluator.evaluate(predictions)
areaUnderPR

0.09633582960376492

In [101]:
# predictions.select('prediction').filter((col("prediction") == 1))
predictions.filter((col("prediction") == 1))

customer_id,terminal_id,tx_amount,tx_time_seconds,tx_fraud,features,scaledFeatures
13,489,89.34,2009974,1,"[13.0,489.0,89.34...",[1.30000130000129...
13,955,111.55,396952,1,"[13.0,955.0,111.5...",[1.30000130000129...
15,61,47.8,3363354,1,"[15.0,61.0,47.8,3...",[1.50000150000149...
15,61,55.96,2897217,1,"[15.0,61.0,55.96,...",[1.50000150000149...
21,81,142.76,4190690,1,"[21.0,81.0,142.76...",[2.1000021000021E...
25,5,25.75,1339276,1,"[25.0,5.0,25.75,1...",[2.50000250000249...
31,640,96.95,1802475,1,"[31.0,640.0,96.95...",[3.10000310000309...
32,298,90.06,826039,1,"[32.0,298.0,90.06...",[3.2000032000032E...
38,705,36.24,2551623,1,"[38.0,705.0,36.24...",[3.80000380000379...
57,940,99.93,330053,1,"[57.0,940.0,99.93...",[5.70000570000569...


### Подбор гиперпараметров

In [243]:
from hyperopt import fmin, rand, tpe, hp, SparkTrials, STATUS_OK

In [244]:
space = {
    "numTrees": hp.choice("numTrees", [100, 200, 300, 400]),
    "maxDepth": hp.quniform("maxDepth", 2, 10, 1),
    "impurity": hp.choice("impurity", ["gini", "entropy"]),
}

In [245]:
def hyperparameter_tuning(params):
    numTrees = params["numTrees"]
    maxDepth = int(params["maxDepth"])
    impurity = params["impurity"]
    
    rf = RandomForestClassifier(labelCol="tx_fraud", featuresCol="features", numTrees=numTrees, 
                                maxDepth=maxDepth,
                                impurity=impurity)
    
    model = rf.fit(training)
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR', labelCol='tx_fraud')
    
    predictions = model.transform(test)
    areaUnderPR = evaluator1.evaluate(predictions)
    
    loss = -areaUnderPR
  
    return {'loss': loss, 'status': STATUS_OK}

In [ ]:
best_spark = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

Другой способ

In [235]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [236]:
rf = RandomForestClassifier(labelCol="tx_fraud", featuresCol="features")

In [237]:
paramGrid = (ParamGridBuilder()
    .addGrid(rf.numTrees, [100, 200])  #, 300, 400
    .addGrid(rf.maxDepth, [2, 7]) #3, 4, 5, 6, 
    .addGrid(rf.impurity, ["gini"])
    .build())

In [238]:
evaluator=BinaryClassificationEvaluator().setLabelCol("tx_fraud")

In [240]:
tvs = TrainValidationSplit(estimator=rf,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8,
                           parallelism=3)

In [242]:
model = tvs.fit(training)

In [ ]:
param_map = model.bestModel.extractParamMap()
for i in param_map:
    print(f"{i.name:<16}\t{param_map[i]}")

In [212]:
# Make predictions on test data. model is the model with combination of parameters
# that performed best.
# model.transform(test)\
#     .select("features", "label", "prediction")\
#     .show()

predictions = model.transform(test)

In [213]:
predictions

customer_id,terminal_id,tx_amount,tx_time_seconds,tx_fraud,features,scaledFeatures,rawPrediction,probability,prediction
0,934,97.76,3092274,0,"[0.0,934.0,97.76,...","[0.0,1.0433644611...",[18.9451642201814...,[0.94725821100907...,0.0
3,205,1.19,3208915,0,"[3.0,205.0,1.19,3...",[3.00000300000299...,[18.9451642201814...,[0.94725821100907...,0.0
12,858,77.16,4649601,0,"[12.0,858.0,77.16...",[1.20000120000119...,[18.8820135576199...,[0.94410067788099...,0.0
18,515,99.54,2523051,0,"[18.0,515.0,99.54...",[1.80000180000179...,[18.9451642201814...,[0.94725821100907...,0.0
19,214,54.2,3037802,0,"[19.0,214.0,54.2,...",[1.90000190000189...,[18.9451642201814...,[0.94725821100907...,0.0
20,413,80.34,1055396,0,"[20.0,413.0,80.34...",[2.00000200000199...,[18.9451642201814...,[0.94725821100907...,0.0
21,401,99.85,5153808,0,"[21.0,401.0,99.85...",[2.1000021000021E...,[18.9023051453165...,[0.94511525726582...,0.0
24,943,104.01,4563025,1,"[24.0,943.0,104.0...",[2.40000240000239...,[18.8996209511319...,[0.94498104755659...,0.0
25,564,47.72,1340442,0,"[25.0,564.0,47.72...",[2.50000250000249...,[18.9451642201814...,[0.94725821100907...,0.0
25,619,5.34,306767,0,"[25.0,619.0,5.34,...",[2.50000250000249...,[18.9451642201814...,[0.94725821100907...,0.0


In [ ]:
# ПРОВЕРИТЬ ТАК
# from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# paramGrid = ParamGridBuilder()\
#     .addGrid(lr.regParam, [0.01, 0.1, 0.5])\
#     .addGrid(lr.fitIntercept, [False, True])\
#     .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
#     .build()

# trainValidationSplit = TrainValidationSplit()\
#     .setEstimator(lr)\
#     .setEvaluator(evaluator)\
#     .setEstimatorParamMaps(paramGrid)\
#     .setTrainRatio(0.7)\
#     .setParallelism(2)

# model = trainValidationSplit.fit(dataF)

# param_map = model.bestModel.extractParamMap()
# for i in param_map:
#     print(f"{i.name:<16}\t{param_map[i]}")

In [100]:
# run_name = 'RF_CVjoblibspark'


    
# with mlflow.start_run(run_name=run_name, experiment_id=experiment.experiment_id):
#     mlflow.log_param('features', FEATURES)
    
#     space = {
#     "n_estimators": [100, 200, 300],
#     "max_depth": range(2, 7),
#     "criterion":["gini", "entropy"],
#     }
#     mlflow.log_param('param_grid', space)
    
# #     model = rf.fit(training)
    
#     parallelism = 4
#     with parallel_backend("spark", n_jobs=parallelism):
#         search = RandomizedSearchCV(
#             estimator=rfsc,
#             param_distributions=space,
#             n_iter=40,
#             cv=2,
#             verbose=2
#         )

#         search.fit(training.select("scaledFeatures"), training.select("tx_fraud"))
    
#     best = search.best_params_
#     best_CV_Score = search.best_score_
    
#     mlflow.log_param('best_params', best)
#     mlflow.log_metric("best_CV_Score", best_CV_Score)
    
    
    
#     predictions = model.transform(test)
#     areaUnderPR = evaluator1.evaluate(predictions)
    
#     mlflow.log_metric("F1", F1)
#     mlflow.spark.save_model(model, "random_forest")

#     mlflow.spark.log_model(model, "random_forest")





ValueError: Expected array-like (array or non-string sequence), got +--------+
|tx_fraud|
+--------+
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
+--------+
only showing top 20 rows


### вторая модель

In [31]:
from pyspark.ml.classification import GBTClassifier

In [47]:
gbt = GBTClassifier(labelCol="tx_fraud", featuresCol="features", maxIter=10)

In [48]:
run_name = 'Gradient-boosted'

with mlflow.start_run(run_name=run_name, experiment_id=experiment.experiment_id):
    mlflow.log_param('features', FEATURES)
    mlflow.log_param('maxIter', 10)
    
    model = gbt.fit(training)
    
    evaluator1 = BinaryClassificationEvaluator(metricName='areaUnderPR', labelCol='tx_fraud')
    evaluator2 = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', 
                                              metricLabel=1, beta=1.0, labelCol='tx_fraud')
    predictions = model.transform(test)
    areaUnderPR = evaluator1.evaluate(predictions)
    F1 = evaluator2.evaluate(predictions)
    
    mlflow.log_metric("F1", F1)
    mlflow.log_metric("areaUnderPR", areaUnderPR)
    mlflow.spark.save_model(model, "GBTClassifier")

    mlflow.spark.log_model(model, "GBTClassifier1")

In [49]:
areaUnderPR

0.13558586390038654

In [58]:
F1

0.0

In [54]:
model.featureImportances

SparseVector(4, {0: 0.0, 1: 0.6656, 2: 0.0, 3: 0.3343})

In [ ]:
evaluator.evaluate